In [1]:
import numpy as np
import pandas as pd
import tqdm
import catboost as cat
from catboost import CatBoostClassifier

In [2]:
%%time
edges = pd.read_csv('./edges.csv')
ids = pd.read_csv('./ids.csv')

Wall time: 2.38 s


### Baseline

Решаем задачу предсказания ребра между двумя вершинами. У нас есть выборка вершин, для которых мы хотим предсказать к каким вершинам можно провести связь. Это легко переформулируется в задачу классификации!

In [6]:
%%time
vertices = pd.read_csv('./vertices.csv', index_col=0) 
vertices['main_okved'] = vertices['main_okved'].astype(str) 
vertices['company_type'] = vertices['company_type'].astype(str) 
vertices['region_code'] = vertices['region_code'].astype(str) 

vertices.head()

Wall time: 1.96 s


,main_okved,region_code,company_type
id,,,
1,46.75,77,Limited
2,41.2,78,Limited
3,25.11,50,Limited
4,45.31,89,Limited
5,56.1,50,Limited


In [7]:
vertices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1534749 entries, 1 to 1534749
Data columns (total 3 columns):
main_okved      1534749 non-null object
region_code     1534749 non-null object
company_type    1534749 non-null object
dtypes: object(3)
memory usage: 46.8+ MB


In [9]:
result = pd.DataFrame(columns=['id_1', 'id_2', 'preds'])

for i in tqdm.tqdm(ids.id): #цикл по всем вершинам из списка для предсказаний
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index() #все вершины, куда есть ребро из текущей
    df2 = edges[edges['id_2'] == i].reset_index() #все вершины, откуда есть ребро в текущую

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']]) #соединяем их
    df['target'] = 1 # они то нам и нужны как вершины куда есть ребро

    df = vertices.join(df.set_index('id_1')['target']).fillna(0) # а в остальные ребер нет, пометим их как 0
    
    X = df[['main_okved', 'region_code', 'company_type']]
    y = df['target']
    
    model = CatBoostClassifier(iterations=30, verbose=False)
    cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)  # собственно, обучаем

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2'] # просто переименование
    
    result = result.append(res, ignore_index=True, sort=False) # записываем результат


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [21:08<00:00, 12.19s/it]

In [11]:
result.sort_values(by='preds', ascending=False)[:100000].to_csv("./submission.csv", index=False)

Accuracy для бейзлайна: 3.41%

### Наше решение

In [13]:
%%time
vertices = pd.read_csv('./data.csv', index_col=0)
embeddings = pd.read_csv("./embeddings(all,lr1e-2,3).csv", index_col=0)
vertices = vertices.join(embeddings)
vertices['main_okved'] = vertices['main_okved'].astype(str) 
vertices['main_okved2'] = vertices['main_okved2'].astype(str) 
vertices['company_type'] = vertices['company_type'].astype(str) 
vertices['fed_district'] = vertices['fed_district'].astype(str)
vertices['letter'] = vertices['letter'].astype(str)
vertices.head()

Wall time: 33.5 s


,main_okved,company_type,n_transactions,value,main_okved2,letter,population,fed_district,0,1,...,22,23,24,25,26,27,28,29,30,31
id,,,,,,,,,,,,,,,,,,,,,
1,46.75,Limited,0.698135,13.898125,46,G,16.341757,Центральный,1.449617e-15,7.116528e-18,...,7.206796e-06,1.845125e-07,1.716499e-06,4.301554e-16,1.633623e-07,8.129408e-14,6.226600e-08,3.162855e-07,8.689434e-18,2.624638e-15
2,41.2,Limited,5.235162,14.052657,41,F,15.492969,Северо-Западный,5.995209e-18,1.237784e-20,...,6.080125e-11,7.671115e-08,8.668347e-10,2.378635e-17,1.877335e-04,5.700303e-17,2.666427e-07,6.160163e-01,2.862846e-20,4.512166e-18
3,25.11,Limited,5.336584,13.381677,25,С,15.830865,Центральный,2.073558e-17,6.927894e-20,...,6.517084e-09,1.751807e-07,5.265328e-09,1.935064e-16,6.917688e-07,5.470254e-16,3.883008e-07,2.954346e-06,2.655530e-19,3.735257e-17
4,45.31,Limited,6.404597,14.527384,45,G,13.196630,Уральский,2.212745e-18,2.421587e-21,...,5.221235e-13,3.165131e-08,4.145046e-10,4.333468e-18,8.346426e-08,1.689583e-17,2.845186e-06,2.972981e-06,5.196674e-21,1.432415e-18
5,56.1,Limited,6.283210,14.628678,56,I,15.830865,Центральный,5.676407e-19,2.285500e-21,...,4.859570e-11,5.042974e-08,5.187225e-10,5.492370e-18,2.901144e-01,2.097201e-17,2.864926e-07,1.746023e-06,4.918666e-21,1.229888e-18


In [ ]:
result = pd.DataFrame(columns=['id_1', 'id_2', 'preds'])
for i in tqdm.tqdm(ids.id): #цикл по всем вершинам из списка для предсказаний
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    np.random.seed(2111091)
    df1 = edges[edges['id_1'] == i].reset_index() #все вершины, куда есть ребро из текущей
    df2 = edges[edges['id_2'] == i].reset_index() #все вершины, откуда есть ребро в текущую

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']]) #соединяем их
    df['target'] = 1 # они то нам и нужны как вершины куда есть ребро

    df = vertices.join(df.set_index('id_1')['target']).fillna(0) # а в остальные ребер нет, пометим их как 0


    X = df.drop(['target'], axis=1) 
    y = df['target']

    model = CatBoostClassifier(iterations=200, # число итераций
                               # 200 оптимальное
                               verbose=50, # через какое число эпох выводить инф-ию о процессе обучения
                               task_type="GPU", # запускаем на гпу, в 4 раза быстрее(классно)
                               # маленький потолок для кодирования категориальных данных
                               score_function = 'NewtonCosine', # функция качества оценки приближения
                               # косинус потому что векторные представления 
                               depth=6, #максимальная глубина деревьев
                               custom_loss=['AUC', 'Accuracy'], # какие параметры выводить в качестве метрики
                               # работает при указании plot=True у метода fit
                               random_state=42, # ну это просто случайный сид
                               per_float_feature_quantization='0:border_count=1200' ) # тут вообще классно, 
                               # это позволяет для самой важной фичи увеличить потолок количества параметров,
                               # в которые ее переводит кэтбуст
    
    cat_features = [0,1, 4,5, 7] # номера категориальных фич, которые кэтбуст сам переведет в числовые признаки для модели

    model.fit(X, y, cat_features)

    X = df.drop(['target'], axis=1)
    preds = model.predict_proba(X)[:,1] # предсказываем для всех ребер с какой вероятностью они нам важны

    df['preds'] = preds # записываем предсказания
    df['id_2'] = i # и указываем вершину, для которой предсказывали

    # возьмем 10000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:10000].reset_index()[['id', 'id_2', 'preds']]
    res.columns = ['id_1', 'id_2', 'preds'] # просто переименование

    result = result.append(res, ignore_index=True, sort=False) # записываем результат


In [ ]:
result.sort_values(by='preds', ascending=False)[100000].to_csv("./submission.csv", index=False)

Accuracy этого решения на привате: 7.99%

### Валидация

По условию, те вершины для которых мы предсказываем  - это крупные компании. Тогда возьмем случайные 15 компаний с большим числом связей с другими.

In [31]:
id_ = np.random.choice(vertices[vertices['n_transactions'] > 13].index, 15, replace=False)
id_

array([ 267283,  506071,  830720,  604125,  552547, 1342335, 1370326,
       1351679, 1293474,  524354,  852594, 1495252,  559269, 1296937,
        707356], dtype=int64)

In [36]:
ans = []
result = pd.DataFrame(columns=['id_1', 'id_2', 'preds'])
edges_count = 0
for i in id_:
    s1 = set(edges[edges['id_1'] == i]['id_2'].dropna().values)
    s2 = set(edges[edges['id_2'] == i]['id_1'].dropna().values)
    s = s1.union(s2) # получили множество вершин, в которые у нас есть ребра
    np.random.seed(42)
    df = vertices.copy()
    df['target'] = np.zeros(df.shape[0])
    
    first_half = list(s)[:int(len(s) * 0.5)] # делим на 2 части, по первой будем обучаться
    second_half = list(s)[int(len(s) * 0.5):] # а по второй валидироваться
    
    df.loc[first_half, 'target'] = 1
    X = df.drop(['target'], axis=1)
    y = df['target']
    model = CatBoostClassifier(iterations=50, verbose=50, task_type="GPU", score_function = 'NewtonCosine',
    depth=6, custom_loss=['AUC', 'Accuracy'], random_state=42, per_float_feature_quantization='0:border_count=1200')
    cat_features = [0, 1, 4, 5, 7]

    model.fit(X, y, cat_features)
    X = df.drop(['target'], axis=1)
    preds = model.predict_proba(X)[:,1]
    df = df.copy()
    df['preds'] = preds
    df['id_2'] = i
    
    res = df.sort_values(by='preds', ascending=False).iloc[:10000].reset_index()[['id', 'id_2', 'preds', 'target']]
    res.columns = ['id_1', 'id_2', 'preds', 'target']
    acc = len(set(res['id_1'].values).intersection(second_half))/(len(second_half))
    print("Model accuracy: ", acc)
    ans.append(acc)
    edges_count += len(second_half)
    result = result.append(res[res['target'] != 1], ignore_index=True, sort=False)
    print("Validation accuracy:", np.array(ans).mean()) # среднее вообще так себе, надо взвешенное



Learning rate set to 0.330049
0:	learn: 0.0964005	total: 125ms	remaining: 6.11s
49:	learn: 0.0083130	total: 6.12s	remaining: 0us
Model accuracy:  0.46678424456202233
Validation accuracy: 0.46678424456202233
Learning rate set to 0.330049
0:	learn: 0.1039771	total: 117ms	remaining: 5.71s
49:	learn: 0.0094724	total: 6.12s	remaining: 0us
Model accuracy:  0.2694283879254977
Validation accuracy: 0.36810631624376
Learning rate set to 0.330049
0:	learn: 0.1402542	total: 113ms	remaining: 5.52s
49:	learn: 0.0250573	total: 6.17s	remaining: 0us
Model accuracy:  0.1615532661609872
Validation accuracy: 0.2992552995495024
Learning rate set to 0.330049
0:	learn: 0.1068615	total: 119ms	remaining: 5.85s
49:	learn: 0.0094316	total: 6.13s	remaining: 0us
Model accuracy:  0.24972657674079474
Validation accuracy: 0.28687311884732547
Learning rate set to 0.330049
0:	learn: 0.0951643	total: 114ms	remaining: 5.57s
49:	learn: 0.0125640	total: 7.08s	remaining: 0us
Model accuracy:  0.4110720562390158
Validation ac

Но это все равно нечестная точность. Ведь мы берем целых 10000 ребер на каждую вершину и не знаем, какие из них попали бы в ответ. Поэтому, возьмем столько ребер, сколько в сумме мы могли бы идеально предсказать.

In [37]:
res = result.sort_values(by='preds', ascending=False)[:edges_count]
for i in res['id_2'].unique():
    s1 = set(edges[edges['id_1'] == i]['id_2'].dropna().values)
    s2 = set(edges[edges['id_2'] == i]['id_1'].dropna().values)
    s = s1.union(s2)
    half = list(s)[int(len(s) * 0.8):]
    edges_ = res[res['id_2'] == i]['id_1'].values
    acc = len(set(res['id_1'].values).intersection(half))/(len(half))
    print("Vertice id:", i)
    print("Edges to predict:", len(half))
    print("Model accuracy: ", acc)

Vertice id: 524354
Edges to predict: 595
Model accuracy:  0.5630252100840336
Vertice id: 1351679
Edges to predict: 9947
Model accuracy:  0.1585402633959988
Vertice id: 552547
Edges to predict: 2276
Model accuracy:  0.6036906854130053
Vertice id: 830720
Edges to predict: 3533
Model accuracy:  0.3189923577696009
Vertice id: 267283
Edges to predict: 1361
Model accuracy:  0.6260102865540044
Vertice id: 707356
Edges to predict: 1215
Model accuracy:  0.34074074074074073
Vertice id: 604125
Edges to predict: 1097
Model accuracy:  0.325432999088423
Vertice id: 1370326
Edges to predict: 1084
Model accuracy:  0.5931734317343174
Vertice id: 506071
Edges to predict: 1246
Model accuracy:  0.43659711075441415
Vertice id: 852594
Edges to predict: 1177
Model accuracy:  0.3135089209855565
Vertice id: 1296937
Edges to predict: 1673
Model accuracy:  0.4512851165570831
Vertice id: 1293474
Edges to predict: 1150
Model accuracy:  0.591304347826087
Vertice id: 1342335
Edges to predict: 849
Model accuracy:  0.